In [ ]:
from huggingface_hub import login

HF_TOKEN = "hf_R*****RkbHvj"
login(token=HF_TOKEN)

In [2]:
!git clone https://github.com/KarthikAvinashFI/bg_noises.git

Cloning into 'bg_noises'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 20 (delta 0), reused 4 (delta 0), pack-reused 15 (from 1)
Receiving objects: 100% (20/20), 169.38 MiB | 15.27 MiB/s, done.
Updating files: 100% (16/16), done.


In [3]:
cd bg_noises/

/content/bg_noises


In [13]:
!git fetch origin

remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 2), reused 25 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (25/25), 73.96 MiB | 6.50 MiB/s, done.
From https://github.com/KarthikAvinashFI/bg_noises
   da541d1..c9d101e  main       -> origin/main


In [14]:
!git pull

Updating da541d1..c9d101e
Updating files: 100% (40/40), done.
Fast-forward
 ambience_busy_office_call_center.wav               | Bin 17159970 -> 0 bytes
 bg_noises_descriptions.json                        |  78 ++++++---
 ...ng_and_wind_through_trees_in_stockwood_park.wav | Bin 16760836 -> 0 bytes
 busy_train_station_ambience.wav                    | Bin 7549588 -> 0 bytes
 crop_audios.ipynb                                  | 178 +++++++++++++++++++++
 ...ng_and_wind_through_trees_in_stockwood_park.wav | Bin 0 -> 7056044 bytes
 .../busy_office_no_people_loop.wav                 | Bin 4259884 -> 3528044 bytes
 .../car_driving_ambience.wav                       | Bin 14332398 -> 10500612 bytes
 final/exterior_ambiance_parking_lot_with_birds.wav | Bin 0 -> 7680044 bytes
 fan_hum.wav => final/fan_hum.wav                   | Bin 1486316 -> 1486146 bytes
 final/grocery_store_ambience_new.wav               | Bin 0 -> 3648044 bytes
 final/office_ambience_mix.wav                      | Bin 0 ->

In [15]:
!ls

audio_descriptions.json      final		     verify_file_names.ipynb
bg_noises_descriptions.json  get_descriptions.ipynb
crop_audios.ipynb	     trimmed_audios


In [16]:
cd final

/content/bg_noises/final


In [17]:
!ls

birdsong_and_wind_through_trees_in_stockwood_park.wav
busy_office_no_people_loop.wav
car_driving_ambience.wav
exterior_ambiance_parking_lot_with_birds.wav
fan_hum.wav
grocery_store_ambience_new.wav
office_ambience_mix.wav
people_at_airport_room.wav
shopping_mall.wav
traffic_jam_in_manila.wav
train_station_ambience_no_trains.wav
train_station_ambience.wav


In [5]:
# Install a transformers version that supports Gemma 3n (>= 4.53)
!pip install 'transformers>=4.53.0' 'timm>=1.0.16' -q

from transformers import AutoModelForImageTextToText, AutoProcessor
import torch
import os
from glob import glob
from IPython.display import display, Markdown


In [6]:
GEMMA_PATH = 'google/gemma-3n-E2B-it'

processor = AutoProcessor.from_pretrained(GEMMA_PATH)
model = AutoModelForImageTextToText.from_pretrained(
    GEMMA_PATH,
    torch_dtype='auto',
    device_map='auto',
)

print(f'Device: {model.device}')
print(f'DType: {model.dtype}')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device: cuda:0
DType: torch.bfloat16


In [33]:
# def describe_audio(path: str, max_tokens: int = 64) -> str:
#     """Run Gemma-3n on a single audio file and return a short description."""
#     system_msg = {
#         'role': 'system',
#         'content': [
#             {
#                 'type': 'text',
#                 'text': (
#                     'You are an assistant that listens to a given background noise audio clip '
#                     'and writes a short, high-level description of what is '
#                     'happening in 1-2 sentences. Mainly describe the sound/noise - how is it and also do mention how loud/unpleasant it is. Based on these descriptions we will be using that audio as background noise when simulating a vapi voice assistant when calling an agent - so that it looks like a real background noise based on the assistants situation. So need the descriptions to select this particular background noise. Start "Heading: "Give a short heading for this background noise"; Description: This noise.."'
#                 ),
#             }
#         ],
#     }

#     user_msg = {
#         'role': 'user',
#         'content': [
#             {
#                 'type': 'text',
#                 'text': (
#                     f"Please describe the following audio file named "
#                     f"'{os.path.basename(path)}' in 1-2 sentences."
#                 ),
#             },
#             {
#                 'type': 'audio',
#                 'audio': path,
#             },
#         ],
#     }

#     messages = [system_msg, user_msg]

#     input_ids = processor.apply_chat_template(
#         messages,
#         add_generation_prompt=True,
#         tokenize=True,
#         return_dict=True,
#         return_tensors='pt',
#     )
#     input_len = input_ids['input_ids'].shape[-1]

#     input_ids = input_ids.to(model.device, dtype=model.dtype)
#     outputs = model.generate(
#         **input_ids,
#         max_new_tokens=max_tokens,
#         disable_compile=True,
#     )

#     decoded = processor.batch_decode(
#         outputs[:, input_len:],
#         skip_special_tokens=True,
#         clean_up_tokenization_spaces=True,
#     )
#     return decoded[0].strip()


def describe_audio(path: str, max_tokens: int = 256) -> str:
    """Run Gemma-3n on a single audio file and return a JSON description object as a string."""
    system_msg = {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": (
                    "You are an assistant that listens to background noise audio clips and describes them.\n\n"
                    "Output ONLY a valid JSON object with no extra text. No code fences, no 'json' label, nothing before or after.\n\n"
                    "Required format:\n"
                    "{\n"
                    '  "title": "Brief descriptive name (e.g., Busy Coffee Shop, Highway Traffic)",\n'
                    '  "description": "2-3 sentences describing the soundscape, including what sounds are present, overall noise level, and character/mood",\n'
                    '  "environment": "one of: office, home, outdoors, transit, retail, vehicle, cafe, street, other",\n'
                    '  "intensity": "one of: quiet, moderate, loud, very_loud"\n'
                    '  "path": "the exact path which is provided to you in the input; AS IT IS PUT IT HERE."\n'
                    "}\n\n"
                    "Guidelines:\n"
                    "- title: Short and descriptive, suitable for UI display\n"
                    "- description: Mention specific sounds (voices, machinery, nature, etc.), how intrusive the noise is, and whether it's continuous or intermittentIt must be like you are suggesting this background noise, than just describing.\n"
                    "- environment: Best-fit category for where this audio would typically occur\n"
                    "- intensity: Based on how loud and disruptive the background noise is\n"
                    "- path: The exact path that is provided to you for the audio file.\n\n"
                    "Output ONLY the JSON object."
                ),
            }
        ],
    }

    user_msg = {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": f"Analyze this audio file and produce the JSON description. This is the path for the audio file: `{path}`.",
            },
            {
                "type": "audio",
                "audio": path,
            },
        ],
    }


    messages = [system_msg, user_msg]

    input_ids = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    )
    input_len = input_ids["input_ids"].shape[-1]

    input_ids = input_ids.to(model.device, dtype=model.dtype)
    outputs = model.generate(
        **input_ids,
        max_new_tokens=max_tokens,
        disable_compile=True,
    )

    decoded = processor.batch_decode(
        outputs[:, input_len:],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True,
    )
    return decoded[0].strip()


In [35]:
# import os
# import json
# from glob import glob

# def describe_audio_directory(directory: str, patterns=None, json_filename="audio_descriptions.json"):
#     """Find all audio files in a directory, describe them, and save to JSON."""
#     if patterns is None:
#         patterns = ('*.wav', '*.mp3', '*.flac', '*.ogg', '*.m4a')

#     paths = []
#     for pattern in patterns:
#         paths.extend(glob(os.path.join(directory, pattern)))

#     paths = sorted(set(paths))
#     if not paths:
#         print(f'No audio files found in {directory}')
#         return []

#     results = []
#     for path in paths:
#         print(f"\n=== {os.path.basename(path)} ===")
#         try:
#             description = describe_audio(path)
#         except Exception as e:
#             print(f'Error describing {path}: {e}')
#             continue
#         print(description)
#         results.append({
#             "path": path,
#             "description": description,
#         })

#     # Save results as JSON
#     json_path = os.path.join(directory, json_filename)
#     with open(json_path, "w", encoding="utf-8") as f:
#         json.dump(results, f, ensure_ascii=False, indent=2)

#     print(f"\nSaved {len(results)} descriptions to {json_path}")
#     return results

import json

def describe_audio_directory(directory: str, patterns=None, json_filename="audio_descriptions.json"):
    """Find all audio files in a directory, describe them, and save normalized JSON objects."""
    if patterns is None:
        patterns = ("*.wav", "*.mp3", "*.flac", "*.ogg", "*.m4a")

    paths = []
    for pattern in patterns:
        paths.extend(glob(os.path.join(directory, pattern)))

    paths = sorted(set(paths))
    if not paths:
        print(f"No audio files found in {directory}")
        return []

    results = []
    for path in paths:
        print(f"\n=== {os.path.basename(path)} ===")
        try:
            raw = describe_audio(path)
        except Exception as e:
            print(f"Error describing {path}: {e}")
            continue

        print("Model raw output:")
        print(raw)
        try:
            obj = json.loads(raw)
        except json.JSONDecodeError as e:
            print(f"Failed to parse JSON for {path}: {e}")
            continue

        # Ensure path is correct, even if the model messed it up
        obj["path"] = path

        results.append(obj)

    # Save results as JSON
    json_path = os.path.join(directory, json_filename)
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"\nSaved {len(results)} descriptions to {json_path}")
    return results


In [36]:
# Change this to another path if needed
audio_dir = '/content/bg_noises/final'

results = describe_audio_directory(audio_dir)
results



=== birdsong_and_wind_through_trees_in_stockwood_park.wav ===
Model raw output:
```json
{
  "title": "Gentle Park Sounds",
  "description": "This audio features calming sounds of birdsong and wind rustling through trees. The overall atmosphere is peaceful and relaxing, creating a serene backdrop. It's a pleasant soundscape perfect for unwinding.",
  "environment": "outdoors",
  "intensity": "quiet",
  "path": "/content/bg_noises/final/birdsong_and_wind_through_trees_in_stockwood_park.wav"
}
```
Failed to parse JSON for /content/bg_noises/final/birdsong_and_wind_through_trees_in_stockwood_park.wav: Expecting value: line 1 column 1 (char 0)

=== busy_office_no_people_loop.wav ===
Model raw output:
```json
{
  "title": "Busy Office Atmosphere",
  "description": "This audio features a consistent hum of office machinery, interspersed with keyboard clicks and faint conversations. The overall soundscape is moderately busy, creating a sense of a typical office environment without any people p

[]

In [37]:
print(json.dumps(results, indent=2))

[]
